# Lets identify a single year that has very little net heatflux.

In [1]:
# !pip install -e /home/jovyan/PROJECTS/ocean_emulators/

In [2]:
import xarray as xr
from ocean_emulators.preprocessing import manual_v0_fixes
from distributed import Client
import hvplot.pandas  # noqa

In [3]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/jbusecke/proxy/8787/status,
Dashboard: /user/jbusecke/proxy/8787/status,Workers: 8
Total threads: 64,Total memory: 503.88 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40791,Workers: 8
Dashboard: /user/jbusecke/proxy/8787/status,Total threads: 64
Started: Just now,Total memory: 503.88 GiB
Comm: tcp://127.0.0.1:45629,Total threads: 8
Dashboard: /user/jbusecke/proxy/43205/status,Memory: 62.99 GiB
Nanny: tcp://127.0.0.1:33681,


In [4]:
path = "gs://leap-persistent/sd5313/input_OM4v0.0"

In [5]:
ds = manual_v0_fixes(xr.open_dataset(path, engine='zarr', chunks={}))
ds_years = ds.resample(time='1YS').mean('time')
hf = ds_years.hfds.load()

In [6]:
df = xr.Dataset({
    'mean': hf.weighted(hf.areacello).mean(['x', 'y']),
    'std':hf.weighted(hf.areacello).std(['x', 'y']),
    'year':xr.DataArray([t.year for t in hf.time.data], dims='time')
}
).to_dataframe()

In [7]:
df.hvplot.scatter(x='std', y='mean', color='year')

:Scatter   [std]   (mean,year)